# Evaluación de la Bolsa de Trabajo - Pregunta electiva B

Para desarrolar la pregunta B de la parte electiva de la evaluación, escogí un dataset de Kaggle basado en opiniones de clientes de Amazon:  [Amazon Reviews](https://www.kaggle.com/bittlingmayer/amazonreviews)

Cada línea del dataset escogido tiene en su estructura las siguientes características:
- Clasificación
- Título
- Contenido

Cabe mencionar que están clasificados como \__label\__1  (negativos) o \__label\__2 (positivos).

Debido a la extensión del dataset (400000 datos) y a las limitaciones en recursos computacionales con las que cuento, sólo trabajaré con 10000 datos, esto se puede personalizar con la variable 'limiteDataSet', cuyo dominio es <0;400000]

### Librerías utilizadas para el problema

In [1]:
import re
from langdetect import detect
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#GaussianNB corresponde a un modelo de clasificación bayesiano
from sklearn.naive_bayes import GaussianNB

### Carga y almacenamiento del dataset

In [2]:
fichero = open('test.ft.txt','r',encoding="utf8")
revisiones = list(map(lambda x:x[:-1], fichero.readlines()))
fichero.close()

In [3]:
categoria, titulo, texto, limiteDataSet = [], [], [], 10000

for line in revisiones[0:limiteDataSet]:
    tupla = re.findall(r"^(__.*__.)(.+?): (.*)", line)
    tupla = tupla[0]
    categoria.append(tupla[0])
    titulo.append(tupla[1][1:])
    texto.append(tupla[2])

del(revisiones)

In [4]:
df = pd.DataFrame(
    {'category': categoria,
     'title': titulo,
     'text': texto
    })

df['review_length'] = df.text.apply(len)
df['title_length'] = df.title.apply(len)

### Filtración y reemplazo de datos no relevantes

Si bien hay que considerar ciertas condiciones del dataset (como caracteres especiales, emoticones que podrían filtrarse, entre otros), sólo se considerará en el filtro de descripciones que no están en inglés.

In [5]:
doc_preprocessed=[]
categorias=[]
filasEnglish=0
for i in range(len(texto)):
    fila = texto[i]
    cat = categoria[i]
    if detect(fila)=='en':
        doc_preprocessed.append(fila)
        categorias.append(cat)
        filasEnglish+=1

### Preparación del modelo TF-IDF

Para realizar la clasificación se utilizará 

In [6]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(doc_preprocessed)

### Entrenamiento de modelo

In [7]:
subCategorias = []
subCategorias.append('__label__1')
subCategorias.append('__label__2')

categorias = np.asarray(subCategorias)

X_data, y_data = X, categorias
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=23)

ValueError: Found input variables with inconsistent numbers of samples: [9971, 2]

In [ ]:
def run_model(clf, X, y):
    scores = cross_val_score(clf, X, y, cv=5)
    accuracy=scores.mean()
    desvEst=scores.std() * 2
    print("%s accuracy: %0.5f (+/- %0.5f)" % \
          (str(clf.__class__).split('.')[-1].replace('>','').replace("'",''), 
          accuracy,desvEst))
    return accuracy,desvEst

In [ ]:
y_train=np.array(y_train)

In [ ]:
print("Rendimiento de entrenamiento")
acc,devEst = run_model(GaussianNB(), X_train, y_train)